# Neural Network 神经网络

## 0 常见的神经网络结构

1. 卷积神经网络(CNN)

   专门用于处理*图像*数据的深度学习模型，*通过卷积层提取特征，并通过池化层降低数据维度*。

2. 循环神经网络(RNN)

   适用于处理*序列*数据，如*时间序列和文本*，*通过内部状态的循环传递来捕捉时序依赖*。

3. 生成对抗网络(GAN)

   由`生成器`（学生）和`判别器`（老师）组成的网络，用于*生成新的数据样本*，广泛应用于*图像生成*（文生图）和*风格转换*。

4. 变换器(Transformer)

   基于`自注意力机制`的模型，用于处理*序列*数据，如自然语言处理中的*文本翻译和生成任务*。

## 1 实战极简神经网络

In [1]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())


2.2.2
True


### 1.1 【原理版】自定义张量的线性回归神经网络模型

In [2]:
# 特征张量（输入）
x = torch.tensor([
    [1, 0, 0],
    [1, 1, 0],
    [1, 0, 1],
    [1, 1, 1]
], dtype=torch.float32)
print(f"特征张量x（输入）：\n{x}")

# 模拟预测参数
w = torch.rand(3, 1)
print(f"预测参数w：\n{w}")

# 模拟预测张量（标签）
y_hat = x @ w
print(f"预测张量y_hat（标签）：\n{y_hat}")


特征张量x（输入）：
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 0., 1.],
        [1., 1., 1.]])
预测参数w：
tensor([[0.7906],
        [0.6718],
        [0.1030]])
预测张量y_hat（标签）：
tensor([[0.7906],
        [1.4624],
        [0.8935],
        [1.5653]])


### 1.2 【框架版】原生封装的线性回归神经网络模型

In [3]:
# 特征张量（输入），去除偏置项
x = torch.tensor([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
], dtype=torch.float32)
print(f"特征张量x（输入），去除偏置项：\n{x}")

# Linear实例参数：
# in_features 上一层神经元的个数（输入特征），out_features 下一层神经元的个数（输出特征）
output = torch.nn.Linear(2, 1)
print(f"线性回归模型的预测张量：\n{output(x)}")
print(f"线性回归模型的参数：\n{output.weight}")
print(f"线性回归模型的偏置项：\n{output.bias}")


特征张量x（输入），去除偏置项：
tensor([[0., 0.],
        [1., 0.],
        [0., 1.],
        [1., 1.]])
线性回归模型的预测张量：
tensor([[-0.1320],
        [-0.8349],
        [-0.0852],
        [-0.7881]], grad_fn=<AddmmBackward0>)
线性回归模型的参数：
Parameter containing:
tensor([[-0.7029,  0.0468]], requires_grad=True)
线性回归模型的偏置项：
Parameter containing:
tensor([-0.1320], requires_grad=True)


## 2 损失函数

每一个样本经过模型后会得到一个预测值，然后得到的预测值和真实值的差值就成为损失（当然损失值越小证明模型越是成功），我们知道有许多不同种类的损失函数，这些函数本质上就是计算预测值和真实值的差距的一类型函数。

很多损失函数的计算结果都与输入特征的范数的值相关。

In [4]:
data = torch.tensor([
    [1, 2, 3, 4],
    [2, 4, 6, 8],
    [3, 6, 9, 12]
], dtype=torch.float32)
res_l2 = torch.norm(data, p=2, dim=1, keepdim=True)
print(f"输入特征：\n{data}")
print(f"L2范数：\n{res_l2}")


输入特征：
tensor([[ 1.,  2.,  3.,  4.],
        [ 2.,  4.,  6.,  8.],
        [ 3.,  6.,  9., 12.]])
L2范数：
tensor([[ 5.4772],
        [10.9545],
        [16.4317]])


常见的损失函数有：

- 平均绝对误差（Mean Absolute Error, MAE），其中 $m$ 为样本数量（列数），$y_i$ 为真实值，$\hat{y}_i$ 为预测值。
    $$\text{MAE} = L_1(y, \hat{y}) = \frac{1}{m} \sum_{i=1}^m |y_i - \hat{y}_i|$$
- 均方误差（Mean Squared Error, MSE），用来衡量模型的预测值和真实值之间的*平均差异*。
    $$\text{MSE} = \frac{1}{m} \sum_{i=1}^m (y_i - \hat{y}_i)^2$$
- 交叉熵（Cross Entropy, CE），主要用于处理`分类`问题，用来衡量模型的预测值和真实值之间的*对数差异*。
    $$\text{CE} = -\frac{1}{m} \sum_{i=1}^m (y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i))$$
    - `分类`任务中，预测值是一些`离散`的概率值（0或1）。特别地，若预测结果和真实情况`一致`，则`交叉熵为0`，否则交叉熵为正值。
    - `回归`任务中，预测值是一些`连续`的概率值。

In [5]:
y = torch.tensor([2, 3, 4, 5], dtype=torch.float32)
y_hat = torch.tensor([4, 5, 6, 7], dtype=torch.float32)
print(f"真实值：\n{y}")
print(f"预测值：\n{y_hat}")

# 继承基类nn，定义损失函数接口loss，并计算损失差异
mae = torch.nn.L1Loss()
print(f"平均绝对误差MAE损失：\n{mae(y, y_hat)}")
mse = torch.nn.MSELoss()
print(f"均方误差MSE损失：\n{mse(y, y_hat)}")
ce = torch.nn.CrossEntropyLoss()
print(f"交叉熵CE损失：\n{ce(y, y_hat)}")


真实值：
tensor([2., 3., 4., 5.])
预测值：
tensor([4., 5., 6., 7.])
平均绝对误差MAE损失：
2.0
均方误差MSE损失：
4.0
交叉熵CE损失：
37.684173583984375


## 3 最优解（局部极值与全局最值）与梯度下降【神经网络的核心命题】

目标函数最优解问题解法：

1. 求导数法：
   简单函数的最优解就是函数的导数为0的点。
2. 梯度下降法【重点】（基于`循环选代`式优化方法）：
   复杂函数的最优解就是函数的梯度方向的负方向。
3. 随机梯度下降法：
   随机梯度下降法是梯度下降法的一种改进版本，每次迭代时只选取一个样本来计算梯度，从而减少计算量。
4. 随机梯度下降法（动量）：
   随机梯度下降法的改进版本，在计算梯度时考虑之前的梯度，从而加速收敛速度。

设函数 $f(x)$ ，求 $f(x)$ 的最小值点 $x$，可根据如下梯度下降法来求解最优解 $x$ 收敛于常数，其中 $\eta$ 为学习率（步长）：
$$
\begin{aligned}
x_{n+1} &= x_n - \eta \nabla f(x_n) \\
&= x_n - \eta \frac{\partial f(x_n)}{\partial x_n}
\end{aligned}
$$

学习率（Learning Rate）是机器学习和深度学习中的一个关键超参数，它决定了在梯度下降优化过程中模型参数更新的步长。一个合适的学习率可以帮助模型快速收敛到最优解，而不适当的学习率可能导致模型训练不稳定（振荡）、收敛速度慢或无法收敛到最优解。


## 4 反向传播算法【神经网络的核心思想】

![前向传播.png](assets/imgs/前向传播.png)

当神经网络完成了一次前向传播的全过程之后，要计算最终输出的预测值与实际值之间的损失，来更新权重，从而最小化损失差异。

反向传播算法(Back Propagation)是一种用于训练多层神经网络的学习方法，它通过计算损失函数相对于网络权重的梯度来更新权重，以便最小化损失函数。

反向传播算法的关键在于`链式法则`，它允许我们计算复杂函数的`梯度`（求导），即使这些函数是由多个简单函数复合而成的。在实际应角中，反向传播算法是训练深度学习模型的基石，广泛应用于图像识别、语音识别、自然语言处理等领域。

![反向传播算法的基本步骤.png](assets/imgs/反向传播算法的基本步骤.png)

迭代次数可依据对模型的要求自行制定，反向传播算法的迭代过程会在达到预设的迭代次数，或者损失小于某一阈值，即准确率达到预期目标时停止，此时神经网络收敛，能够成功训练出模型。